In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from skimage import morphology, io
from skimage.transform import rescale, resize, downscale_local_mean
from skimage.segmentation import slic, mark_boundaries
from skimage import feature, img_as_ubyte,measure,color,morphology, io, exposure
from skimage.transform import rescale, resize, downscale_local_mean
from skimage import img_as_ubyte,measure,color,morphology, io, exposure
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, roc_curve, roc_auc_score, auc, confusion_matrix, precision_score, pairwise
from imblearn.over_sampling import RandomOverSampler
import os
import cv2
import math
import joblib

#Specify image filepath here
image_filepath = ""
segmentation_filepath = ""

#import images
im = plt.imread(image_filepath)
seg = plt.imread(segmentation_filepath)